## Fast Fourier Transform

This notebook does the following:
* imports a .wav audio file
* displays the signal in the time domain
* converts the signal into the frequency domain
* filters the signal in the frequency domain
* convert the signal back into the time domain and displays it. <br> <br>

This is pretty neat 😀  Hope you enjoy it! (And thanks to Dr. Van Howe for his help!)  <br><br>

---



### Import the audio file

The first couple cells import some needed libraries, and then import the audio file from your Google drive.  Change the `filename` in the third cell to define the folder from which you are importing.  This will take a couple minutes to run.

In [ ]:
#import libraries needed
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio
from scipy.fft import fft, ifft, fftshift,rfft,ifftshift

In [ ]:
!wget https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Sound/C_trumpet_E4.wav
rate, audio = wavfile.read('C_trumpet_E4.wav')


We've imported a file that contains the sound of a trumpet playing a middle E.  Take a listen!

In [ ]:
wavfile.write('audin.wav', rate, audio)
Audio('audin.wav')

Notice that this audio file is a discretized wave signal: that is, it is an array of data that represents the amplitude of the wave at different moments in time.  In other words, it's just a one-dimensional list of numbers.  Here we can see the first 100 elements:

In [ ]:
print(audio[0:100])

### Display the signal in the time domain

Let's take a closer look at the discretized version of this signal.  To do so, we want to define and prints out some key information about the file:

In [ ]:
# Define the length of the array
N = len(audio)

# The sampling rate 'rate' is determined when the audio file
# is recorded; here that is converted into the time step size 'dt'
dt = 1/rate

# Define the total time and the frequency resolution
total = N/rate
df = 1/total

# This makes an index that goes from -half the pts to +half the pts in the audio file
# The center is 0 now, and this makes the math for the FFT work better
# This is then transformed into arrays for time and frequency
x = np.linspace(-N/2,N/2-1,N)
time = dt*x
freq = df*x

# Print out the key values
print("N = ", N)
print("Rate = ", rate)
print("Dt = ",  dt)
print("Total = ",  total)
print("df = ",  df)

✅ ✅ Before going on, answer the "Set 1" questions on the worksheet.

In [ ]:
# plot the audio file as the waveform
plt.figure()
plt.plot(time, audio)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude)');
# The next line changes the range of the x-axis
# Uncomment and change the arguments to zoom in on the signal
plt.xlim(0,0.02);

✅ ✅ Before going on, answer the "Set 2" questions on the worksheet.

### Convert the signal to the frequency domain

In the time domain, the x-axis is in units of time.  The FFT transforms the signal--much like changing a coordinate system--so that the x-axis is in units of frequency.  This allows us to see which frequency sine waves are added together to get the complex signal that we started with.

In [ ]:
#  Now we'll do some Fourier magic!!!
yf=fftshift(fft(audio))

# Plot the Fast Fourier Transform (FFT)
# Each spike represents a sine wave
plt.figure()
plt.plot(freq, abs(yf))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude)')
# Change the line below to zoom in on relevant data
plt.xlim(0,1000);

✅ ✅ Before going on, answer the "Set 3" questions on the worksheet.

### Filtering the signal in the frequency domain

Filtering in the frequency domain is pleasingly simply: all we need to do is multiply the frequencies that we want to keep by 1.0, and the frequencies that we want to filter by 0.   We'll set this up so that it can be a low-pass or a high pass filter. <br><br>

You only need to change the variables in this first cell to control the filter:

In [ ]:
# Define whether you want a high pass or low pass filter.
# If you want a high pass filter, define 'hipass' as 1;
# for a low pass filter, define it as zero
hipass = 0

# Define the cutoff frequency (in Hz)
cutoff = 500

We're going to make an array that includes only 1s and 0s that is the same size as the frequency domain audio file.   By multiplying that file element-wise with this array, we will keep some frequencies and eliminate others.  The image shows how element-wise multiplication works between two arrays:

<img src = https://github.com/AugustanaPEA/ENGR_290/raw/main/Images/Element_wise_mult.PNG width = 300>

In [ ]:
# Convert the cutoff frequency from Hz to #pts in the array
edge = cutoff/df

# Define the data points that are inside the filter
# There is a high and a low cutoff because the FFT produces
# positive and negative frequencies (that mirror each other)
range_lo=round(N/2-edge)
range_hi=round(N/2+edge)

# Now we make our filter array, depending on the value of 'hipass'
if hipass == 1:
    yfilt=np.ones(len(freq))
    yfilt[range_lo:range_hi]=0
else:
    yfilt=np.zeros(len(freq))
    yfilt[range_lo:range_hi]=1

Now we'll plot out the FFT with a red line showing us which frequencies will be kept and which will be filtered out:

In [ ]:
# Plot the FFT and the filter window
# to see if our filter is in the correct place
plt.figure()
plt.plot(freq, abs(yf))
plt.plot(freq,yfilt*max(abs(yf)),'-r')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude)')
plt.xlim(0,2000);

Finally, we'll multiply the full frequency spectrum `yf` by the filter array in order to get a new filtered spectrum called `yfnew`.  The plot shows the spectrum of the filtered signal: we've removed the frequencies either above or below the cutoff frequency:

In [ ]:
###time to do the filtering, just a simple multiplication in the freq domain
yfnew=yf*yfilt

###plot filtered spectrum; should be just what we let in
plt.figure()
plt.plot(freq, abs(yfnew))
plt.plot(freq,yfilt*max(abs(yfnew)),'-r')
plt.xlabel('Frequency (kHz)')
plt.ylabel('Amplitude)')
plt.xlim(0,2000);

✅ ✅ Before going on, answer the "Set 4" questions on the worksheet.

### Convert the signal back to the time domain

Finally, we want to see what the filtered signal looks like as a wave.  So we use the Inverse Fast Fourier Transform: this converts our frequency domain description of the filtered wave (that is, the plot just above this cell) into a time-domain description of the signal. <br><br>

To be able to see what the filter did to the signal, the filtered signal is printed over the unfiltered signal.  Play with the filter parameters to see the effect of different filters!

In [ ]:
####transform back into time domain with inverse fft or ifft
yt=ifft(ifftshift(yfnew))


###Filtered wav in time domain
plt.figure()
plt.plot(time, np.real(yt),'-r')
plt.plot(time, audio, '--')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude)')
plt.legend(["Filtered Signal", "Unfiltered Signal"], loc ="upper right")
# Play with the line below to zoom in or out
plt.xlim(0,0.02);

You can see the difference between the filtered and unfiltered signal, but you can also *hear* that difference if we convect the filtered signal back into an audio `.wav` file:

In [ ]:
# 'yt' is actually an array of complex numbers; we just want the real part
audiout=np.real(yt)

# Now we can convert the file back into a .wav file and play it
audiout=np.asarray(audiout, dtype=np.int16)
wavfile.write('audiout.wav', rate, audiout)
Audio('audiout.wav')

In [ ]:
# Now compare it to the original unfiltered signal
Audio('audin.wav')

✅ ✅ Now play with the filter a bit, and answer "Set 5".

### Use a filter and fourier analysis on your own voice

Now we'll try the above process on a sound signal that you create.  To do this, you'll need to use an app called DecibelX.  **Note that this last step is here mostly because it's really interesting to see a voice represented as a sound signal.  But if you get stuck on a technical point in recording or uploading your clip, you do not need to spend a lot of time debugging: I will consider the assignment complete if you finish through step 5.

<br>

Follow these steps:

1. Go to the App Store on your phone, and download the free DecibelX app.   You don't need to any advanced features, so no need to upgrade.

2. You can play around with the app if you'd like.  Notice that it provides an instantaneous representation of sound in the frequency domain on the opening page (that is, frequency is on the x-axis).  You can slide the top screen (the plot) to the left, and see other representations of the sound.

3. You want to record a clip of your voice singing or humming a constant note.  So first, clear the memory: press "pause", and then the circular arrow just next to the pause/play button.  Click "OK" to reset the recording data.

4. Now you will record the clip.  Start singing/ humming a constant note.  Try to keep the amplitude and note as consistent as possible.  While singing, click "play" and keep singing for about 10 seconds.  While still singing, click "pause" and stop singing (if you'd like : )

5. Now upload the clip to your computer.  Click the download arrow to the right of the "play" button, and click "save" (just keep the name "Record 1").  Then click the "Data" tab at the bottom of the screen.   You should see your "Record 1" listed on the Data page.   Click the lines-and-dots graphic in the upper right, which will allow you to choose a recording: check the circle that appeared next to "Record 1".

6. Now click the "upload" arrow icon at the top of the screen.  Choose "wav" as the type of format.  Now choose to upload to Google Drive (or email if that is easier), and click "Upload".  Sign in as necessary.

7. Now go to your Google Drive.  You should see a zipped Decibel X folder.  Open that, and "extract all" the files.  Move the "Record 1" file to the main folder of your Google Drive (that is, in "My Drive").

8. Look at the file extension.  If it is in .WAV format, you are good: that is what we want, so no need to change this.  If you used an iPhone, however, the format will be .MP4 or .M4A.  In this case (or if the file is anything but a .WAV file), go to a converter website like https://cloudconvert.com/ and convert the file to a .WAV file, and download the converted file.  Move the downloaded file from your Downloads to your My Drive.  

9. You should now have a .WAV file in the My Drive folder of your Google Drive.   For convenience, shorten the name of the file: "record_1.wav" would do fine.

10. Now upload the file into Colab.  Check that the file name and location match the code below, and run these two cells:

In [ ]:
# Grant Colab acces to your google drive
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# Read an audio wavfile
# Replace the file address here with the appropriate address
filename = '/gdrive/MyDrive/record_1.wav'
rate, audio_voice = wavfile.read(filename)

We have to make some changes to the format of the .wav file because it differs a bit from the trumpet file we used above.  We'll name the new file "audio" so we don't have to change any more code above.

In [ ]:
n = len(audio_voice)
audio = np.zeros((n,), dtype=int)
for i in range(n):
    audio[i] = audio_voice[i][0]

In [ ]:
# Allow the file to be played in Colab
wavfile.write('audin.wav', rate, audio_voice)
Audio('audin.wav')

Ok, now your voice file is ready to be analyzed and filtered!   Go back to the beginning of the notebook, and run the cells after the "Display the signal in the time domain."  DO NOT "Run All", or you will overwrite the new "audio" file with the trumpet file.

✅ ✅ Now play with the filter on your voice file, looking at the form and sound of the filtered signal, and answer "Set 6".